# Brain Tumor Detection
## Single Task Validation - Building multi-lable binary Classifier
Description
This dataset was originally created by Yousef Ghanem. To see the current project, which may have been updated since this version, please go here: https://universe.roboflow.com/yousef-ghanem-jzj4y/brain-tumor-detection-fpf1f.

This dataset is part of RF100, an Intel-sponsored initiative to create a new object detection benchmark for model generalizability.

Access the RF100 Github repo: https://github.com/roboflow-ai/roboflow-100-benchmark

## Imports

In [1]:
# Go to project root folder
import os
os.chdir("../")
%pwd

'/workspaces/brain-tumor-detection'

In [2]:
import numpy as np
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

import tensorflow as tf
tf.random.set_seed(42)

import matplotlib.pyplot as plt

2025-03-14 13:02:27.970467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741957347.979432  282792 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741957347.981931  282792 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1741957347.993053  282792 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741957347.993067  282792 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1741957347.993069  282792 computation_placer.cc:177] computation placer alr

In [3]:
found_gpu = tf.config.list_physical_devices('GPU')
if not found_gpu:
    raise Exception("No GPU found")
found_gpu, tf.__version__

([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.19.0')

In [4]:
from src.data_handler.data_loader import DataLoader
from src.data_handler.annotation_processor import AnnotationProcessor
from src.data_handler.preprocessor import Preprocessor

In [5]:
# auto reload dotenv 
%load_ext dotenv
%dotenv

# auto reload libs
%load_ext autoreload
%autoreload 2

## Paths Setup

In [6]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/-Brain-Tumor-Detection-2/train/


In [7]:
cfg.DATASET_DIRS

{'TRAIN_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/train/', 'VALIDATION_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/valid', 'TEST_DIR': '${DATASET.DATASET_DIR}/${DATASET.DATASET_NAME}/test'}

In [8]:
DATASET_DIRS = Path(cfg.DATASET.DATASET_DIR)
TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
VALIDATION_DIR = Path(cfg.DATASET_DIRS.VALIDATION_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)


IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE
LOG_DIR = cfg.OUTPUTS.LOG_DIR
CHECK_POINT_DIR = Path(cfg.OUTPUTS.CHECKPOINT_PATH)
CLASS_NAME = [
    'label0',
    'label1',
    'label2'
]
class_map = {k: v for k, v in enumerate(CLASS_NAME)}

NUM_EPOCHS = cfg.TRAIN.NUM_EPOCHS
LEARNING_RATE = cfg.TRAIN.LEARNING_RATE

NUM_CLASSES = len(CLASS_NAME)


## Dataset Download from Roboflow

In [9]:
if not TRAIN_DIR.exists():
#     from roboflow import Roboflow
#     rf = Roboflow()
#     project = rf.workspace("roboflow-100").project("brain-tumor-m2pbp")
#     version = project.version(2)
#     dataset = version.download("tensorflow") 

    from roboflow import Roboflow
    rf = Roboflow(api_key="AAjLIN3PenSZ29LjbI3d")
    project = rf.workspace("yousef-ghanem-jzj4y").project("brain-tumor-detection-fpf1f")
    version = project.version(2)
    dataset = version.download("tensorflow")
                        

## Load images from directory

### Load Training datasets

In [10]:
prepare_train_dataset = AnnotationProcessor(annotation_file= str(TRAIN_DIR/'_annotations.csv'))
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
train_images, train_class_ids, train_bboxes  = prepare_train_dataset.process_annotations(image_dir=TRAIN_DIR, class_id_map=_class_map)

len(train_images), len(train_class_ids), len(train_bboxes)

(6851, 6851, 6851)

In [11]:
train_class_ids

[[0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1],
 [0, 1, 2],
 [0, 1, 2],
 [1, 2],
 [0, 1],
 [1],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [1, 2],
 [0, 1],
 [2],
 [1],
 [0, 1, 2],
 [1],
 [1],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1],
 [1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1],
 [0, 1, 2],
 [1],
 [0, 1],
 [0, 1, 2],
 [1],
 [1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [1],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2],
 [1],
 [0, 1, 2],
 [0, 1, 2],
 [0, 1, 2]

In [12]:
train_images[0],train_class_ids[0], train_bboxes[0]

('datasets/-Brain-Tumor-Detection-2/train/volume_201_slice_92_jpg.rf.0012bdce7561c3808ed18953dc75060c.jpg',
 [0, 1, 2],
 array([[0.68345324, 0.54545455, 0.95683453, 0.76515152],
        [0.42446043, 0.48484848, 0.99280576, 0.96969697],
        [0.46043165, 0.53030303, 0.99280576, 0.78030303]]))

In [13]:
train_dl = DataLoader(train_images, train_class_ids, train_bboxes)
train_ds = train_dl.load_train_dataset()
train_ds = Preprocessor(train_ds).preprocess()
train_ds = train_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

I0000 00:00:1741957362.483815  282792 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7144 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


In [14]:
for batch in train_ds.take(1):
    image, cls = batch
    print(image.shape, cls.shape)
    print(cls[5])
    print(image[1].numpy().min(), image[1].numpy().max())
    for c in cls:
        print(c.numpy())

(32, 240, 240, 3) (32, 3)
tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)
-123.68 107.654106
[0. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[1. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[0. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 0.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]
[1. 1. 1.]
[1. 1. 1.]
[0. 1. 0.]


2025-03-14 13:02:49.504794: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Validation datasets 

In [15]:
prepare_valid_dataset = AnnotationProcessor(annotation_file= str(VALIDATION_DIR/'_annotations.csv'))

valid_image_paths, valid_class_ids, valid_bboxes  = prepare_valid_dataset.process_annotations(image_dir=VALIDATION_DIR, class_id_map=_class_map)
len(valid_image_paths), len(valid_class_ids), len(valid_bboxes)

(1963, 1963, 1963)

In [16]:
valid_dl = DataLoader(valid_image_paths, valid_class_ids, valid_bboxes).load_val_dataset()
valid_ds = Preprocessor(valid_dl).preprocess()
valid_ds = valid_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

In [17]:
for batch in valid_ds.take(1):
    image, cls, = batch
    print(cls.shape)
    print(image.shape)
    print(image[1].numpy().min(), image[1].numpy().max())

(32, 3)
(32, 240, 240, 3)
-123.68 138.49847


2025-03-14 13:02:50.830981: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Training Setup

In [18]:
METRICS = [
    'binary_accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='AUC', multi_label=True), 
    tf.keras.metrics.F1Score(name='f1_score',average='weighted'),
]

### Define  Callbacks

In [19]:
import os

to_monitor = 'val_AUC'
mode = 'max'
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, 
                                            patience=5, 
                                            monitor=to_monitor,
                                            mode=mode,
                                            min_lr=1e-8,
                                            verbose=1),

    tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(str(CHECK_POINT_DIR), "classifier_ckpt_{epoch}.keras") ,
                                        save_weights_only=False,
                                        save_best_only=True,
                                        monitor=to_monitor,
                                        mode=mode,
                                        verbose=1),
                                        
    tf.keras.callbacks.EarlyStopping(monitor=to_monitor, 
                                    patience=15,
                                    mode=mode, 
                                    restore_best_weights=True,
                                    verbose=1),

]

## Define loss

In [20]:
padded_class_ids = train_dl.multi_hot_class_ids
padded_class_ids[:10]

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [0., 1., 0.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 0.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [21]:
from src.losses import binary_weighted_loss as _loss

positive_weights, negative_weights = _loss.compute_class_weights(padded_class_ids)
positive_weights, negative_weights 

(array([0.38140417, 0.01547219, 0.40767771]),
 array([0.61859584, 0.9845278 , 0.5923223 ], dtype=float32))

### Define Optimizer

In [22]:
optimizer=tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE)

## Define ResNet50 Model Classifier

In [23]:
from src.models.resnet101 import resnet101_classifier
tf.keras.backend.clear_session()

model = resnet101_classifier(input_shape=(IMG_SIZE,IMG_SIZE,3), num_classes=NUM_CLASSES)

model.summary()

Total layers in ResNet101: 345
Unfreezing the last 172 layers (50% of total layers)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 240, 240,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 246, 246,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 120, 120,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 120, 120,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 120, 120,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 122, 122,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 60, 60,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 60, 60,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 60, 60,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 60, 60,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 60, 60,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 60, 60,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 60, 60,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 60, 60,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 60, 60,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 60, 60,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 60, 60,    │      1,024 │ conv2_block1_3_c

 Total params: 44,759,427 (170.74 MB)

 Trainable params: 32,739,331 (124.89 MB)

 Non-trainable params: 12,020,096 (45.85 MB)

## Model Building and Compilation

In [24]:
model.compile(
    optimizer=optimizer,
    loss=_loss.set_binary_crossentropy_weighted_loss(positive_weights, negative_weights),
    metrics=METRICS)  

## Train and Validate the model

In [25]:
import keras

class LearningRateFinder(keras.callbacks.Callback):
    def __init__(self, start_lr=1e-6, end_lr=1e-1, num_epochs=5):
        super().__init__()
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.num_epochs = num_epochs
        self.lrs = []
        self.losses = []

    def on_train_begin(self, logs=None):
        self.lrs = []
        self.losses = []
        self.iteration = 0
        self.lr = self.start_lr

    def on_train_batch_end(self, batch, logs=None):
        self.lrs.append(self.lr)
        self.losses.append(logs['loss'])
        self.iteration += 1

        # Calculate the new learning rate (linear increase for simplicity)
        self.lr = self.start_lr + (self.end_lr - self.start_lr) * (self.iteration / (self.params['steps'] * self.num_epochs))
        # Update the learning rate (handle SGD and Adam)

        optimizer.learning_rate.assign(self.lr)


    def plot_lrs(self):
        import matplotlib.pyplot as plt
        plt.plot(self.lrs, self.losses)
        plt.xscale('log')
        plt.xlabel('Learning Rate')
        plt.ylabel('Loss')
        plt.show()

In [26]:
# Assuming your model, optimizer, and data are already defined

# Instantiate the LearningRateFinder callback
# lr_finder = LearningRateFinder(start_lr=1e-6, end_lr=1e-1, num_epochs=5) # Adjust num_epochs as needed

# # Train the model with the callback
# model.fit(
#     train_ds, # your training data
#     epochs=lr_finder.num_epochs,
#     callbacks=[lr_finder]
# )

# # Plot the results
# lr_finder.plot_lrs()

In [ ]:
import mlflow

mlflow.set_experiment("/brain-tumor-resnet50_classifier")
mlflow.tensorflow.autolog(log_models=True, 
                          log_datasets=False, 
                          log_input_examples=True,
                          keras_model_kwargs={"save_format": "keras"},
                          checkpoint_monitor=to_monitor, 
                          checkpoint_mode=mode)

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=valid_ds,
    batch_size=BATCH_SIZE,
    callbacks=[callbacks],
)

2025/03/14 13:02:55 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.18.0, but the installed version is 2.19.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.
2025/03/14 13:02:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '5312d8cc59b6451188a86fc8000cc030', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/50


I0000 00:00:1741957401.819879  283058 service.cc:152] XLA service 0x7dea1c003570 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741957401.819957  283058 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2025-03-14 13:03:22.581822: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741957406.092438  283058 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1741957416.360524  283058 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - AUC: 0.6381 - binary_accuracy: 0.6083 - f1_score: 0.5086 - loss: 0.6986 - precision: 0.8170 - recall: 0.5960
Epoch 1: val_AUC improved from -inf to 0.81360, saving model to output/checkpoints/classifier_ckpt_1.keras


215/215 ━━━━━━━━━━━━━━━━━━━━ 91s 235ms/step - AUC: 0.6385 - binary_accuracy: 0.6086 - f1_score: 0.5087 - loss: 0.6983 - precision: 0.8172 - recall: 0.5962 - val_AUC: 0.8136 - val_binary_accuracy: 0.6570 - val_f1_score: 0.5193 - val_loss: 0.5660 - val_precision: 0.8992 - val_recall: 0.6068 - learning_rate: 1.0000e-04
Epoch 2/50
214/215 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - AUC: 0.8066 - binary_accuracy: 0.7175 - f1_score: 0.5530 - loss: 0.5468 - precision: 0.9062 - recall: 0.6816
Epoch 2: val_AUC improved from 0.81360 to 0.85596, saving model to output/checkpoints/classifier_ckpt_2.keras


215/215 ━━━━━━━━━━━━━━━━━━━━ 37s 146ms/step - AUC: 0.8069 - binary_accuracy: 0.7177 - f1_score: 0.5530 - loss: 0.5466 - precision: 0.9063 - recall: 0.6819 - val_AUC: 0.8560 - val_binary_accuracy: 0.6920 - val_f1_score: 0.5766 - val_loss: 0.5561 - val_precision: 0.9655 - val_recall: 0.6075 - learning_rate: 1.0000e-04
Epoch 3/50


In [ ]:
# model.compile(
#     optimizer=tf.keras.optimizers.AdamW(learning_rate=1.0000e-05),
#     loss= _loss.set_binary_crossentropy_weighted_loss(positive_weights, negative_weights),
#     metrics=METRICS) 

# history_fine = model.fit(
#     train_ds,
#     epochs=NUM_EPOCHS+40,
#     initial_epoch=history.epoch[-1], # start from last epoch
#     validation_data=valid_ds,
#     batch_size=BATCH_SIZE,
#     callbacks=[callbacks],
# )

## Training Visualization

In [ ]:
import matplotlib.pyplot as plt

def visualize_training_results(history):
    """
    Visualizes training and validation loss, and training and validation accuracy.

    Args:
        history: A dictionary or object containing training history data.
                 For example, a Keras History object or a dictionary with keys:
                 'loss', 'val_loss', 'accuracy', 'val_accuracy'.
    """

    if isinstance(history, dict):
        # Assumes history is a dictionary
        loss = history.get('loss')
        val_loss = history.get('val_loss')
        accuracy = history.get('accuracy')
        val_accuracy = history.get('val_accuracy')
    else:
        # Assumes history is a Keras History object or similar
        loss = history.history.get('loss')
        val_loss = history.history.get('val_loss')
        accuracy = history.history.get('accuracy')
        val_accuracy = history.history.get('val_accuracy')

    if loss and val_loss:
        epochs = range(1, len(loss) + 1)

        plt.figure(figsize=(12, 5))

        # Plot training & validation loss values
        plt.subplot(1, 2, 1)
        plt.plot(epochs, loss, 'r', label='Training loss')
        plt.plot(epochs, val_loss, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

    if accuracy and val_accuracy:
        if not (loss and val_loss):
          plt.figure(figsize=(12, 5))
        else:
          plt.subplot(1, 2, 2)
        # Plot training & validation accuracy values
        plt.plot(epochs, accuracy, 'r', label='Training accuracy')
        plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

    plt.tight_layout() #prevents overlapping titles/labels
    plt.show()


In [ ]:
visualize_training_results(history.history)

## Model Evaluation

### Testing Datasets setup

In [ ]:

prepare_test_dataset = AnnotationProcessor(annotation_file= str(TEST_DIR/'_annotations.csv'))
_class_map = {v: k for k, v in enumerate(CLASS_NAME)}
test_image_paths, test_class_ids, test_bboxes = prepare_test_dataset.process_annotations(image_dir=TEST_DIR, class_id_map=_class_map)

len(test_image_paths), len(test_class_ids), len(test_bboxes)

In [ ]:
test_dl = DataLoader(test_image_paths, test_class_ids, test_bboxes, img_size=IMG_SIZE)
test_ds = test_dl.load_val_dataset()
test_padded_class_ids = test_dl.pad_cls_id_bbx()[0]
test_ds = Preprocessor(test_ds).preprocess()
test_ds = test_ds.batch(BATCH_SIZE)\
                .prefetch(tf.data.AUTOTUNE)

In [ ]:
transformer = Binarizer().fit(test_padded_class_ids) 
y_true = transformer.transform(test_padded_class_ids)
y_true

In [ ]:
results = model.evaluate(test_ds,return_dict=True)
mlflow.log_metrics(results)
print("Testing accuracy: ", results)

In [ ]:
from sklearn.metrics import classification_report

y_prob_pred = model.predict(test_ds)

In [ ]:
y_pred = (y_prob_pred>0.5).astype(int)
y_pred

In [ ]:
report = classification_report(y_true, y_pred, labels=[0,1,2], target_names=CLASS_NAME)
print(report)

In [ ]:
import json

report_json = json.dumps(report, indent=2)
with open("classification_report.json", "w") as f:
    f.write(report_json)

mlflow.log_artifact("classification_report.json")


In [ ]:
from sklearn import metrics

auc_roc_values = []
fig, axs = plt.subplots(1)
for i in range(len(CLASS_NAME)):
    try:
        roc_score_per_label = metrics.roc_auc_score(y_true=y_true[:,i], y_score=y_prob_pred[:,i])
        auc_roc_values.append(roc_score_per_label)
        fpr, tpr, _ = metrics.roc_curve(y_true=y_true[:,i],  y_score=y_prob_pred[:,i])
        
        axs.plot([0,1], [0,1], 'k--')
        axs.plot(fpr, tpr, 
                label=f'{CLASS_NAME[i]} - AUC = {round(roc_score_per_label, 3)}')

        axs.set_xlabel('False Positive Rate')
        axs.set_ylabel('True Positive Rate')
        axs.legend(loc='lower right')
    except:
        print(
            f"Error in generating ROC curve for {CLASS_NAME[i]}. "
            f"Dataset lacks enough examples."
        )
plt.savefig(f"{cfg.OUTPUTS.OUPUT_DIR}/ROC-Curve.png")
mlflow.log_figure(fig, 'ROC-Curve.png')
